In [1]:
import numpy as np
import pandas as pd

 - **Layers**: an Integer value representing the total number of hidden layers in the network (input and output layers are extra).
 
 - **Nodes**: an integer array of size [0,..,Layers+1] containing the dimensions of the neural
network. Nodes[0] shall represent the input size (typically, 50), Nodes[Layers+1]
shall represent the number of output nodes (typically, 1). All other values Nodes[i]
represent the number of nodes in hidden layer i.

 - **NNodes**: a possible alternative to the Nodes parameter for situations where you want
each hidden layer of the neural network to be of the same size. In this case, the size of
the output layer is assumed to be 1, and the size of the input layer can be inferred from
the dataset.

 - **Activations**: an array of size [0,..,Layers+1] (for the sake of compatibility) in which
Activations[0] and Activations[Layers+1] are not used, while all other
Activations[i] values are labels indicating the activation function used in layer i.
This allows you to build neural networks with different activation functions in each layer.

 - **ActivationFn**: a possible alternative to Activations when all hidden layers of your neural
network use the same activation function.

In [2]:
class NeuralNetwork:
    #Layers: an Integer value representing the total number of hidden layers in the network 
    #        (input and output layers are extra)

    def __init__(self, Layers, Nodes, NNodes, Activations, ActivationFn):
        self.Layers = Layers
        self.Nodes = Nodes
        self.NNodes = NNodes
        self.Activations = Activations
        self.ActivationFn = ActivationFn

In [3]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

In [4]:
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [5]:
def ReLu(e):
    return max(0,e)

In [6]:
def leakyRelu(e,a=0.1):
    if e > 0:
        return e
    else:
        return a*e

In [7]:
def sigmoid(e):
    return 1/(1+np.exp(1)**-e)

In [8]:
def tanh(e):
    return 2*sigmoid(2*e) - 1

In [9]:
df = iris.drop('species',axis=1)
# b = petal_width

In [10]:
w = np.array(df.mean())
w = [list(w)]*3
w = np.array(w)
w

array([[5.84333333, 3.05733333, 3.758     , 1.19933333],
       [5.84333333, 3.05733333, 3.758     , 1.19933333],
       [5.84333333, 3.05733333, 3.758     , 1.19933333]])

In [11]:
one = np.array(df.iloc[0])
one[len(one)-1] = 1
one

array([5.1, 3.5, 1.4, 1. ])

In [12]:
h = w.dot(one)
h

array([46.9622, 46.9622, 46.9622])

In [13]:
pd.Series(h).apply(tanh)

0    1.0
1    1.0
2    1.0
dtype: float64

In [14]:
h = pd.Series(h).apply(leakyRelu,args=(0.2,))
h

0    46.9622
1    46.9622
2    46.9622
dtype: float64

In [15]:
b2 = 2
w2 = np.array([1,2,3,b2])
z = w2.dot(np.append(h,1))

In [16]:
z

283.7732000000001

In [18]:
df["petal_width"] = 1

In [19]:
df

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,1
1,4.9,3.0,1.4,1
2,4.7,3.2,1.3,1
3,4.6,3.1,1.5,1
4,5.0,3.6,1.4,1
...,...,...,...,...
145,6.7,3.0,5.2,1
146,6.3,2.5,5.0,1
147,6.5,3.0,5.2,1
148,6.2,3.4,5.4,1


In [20]:
w

array([[5.84333333, 3.05733333, 3.758     , 1.19933333],
       [5.84333333, 3.05733333, 3.758     , 1.19933333],
       [5.84333333, 3.05733333, 3.758     , 1.19933333]])

In [23]:
w.T

array([[5.84333333, 5.84333333, 5.84333333],
       [3.05733333, 3.05733333, 3.05733333],
       [3.758     , 3.758     , 3.758     ],
       [1.19933333, 1.19933333, 1.19933333]])

In [30]:
h = df.dot(w.T).applymap(ReLu)
#df.dot(w.T)
h

,0,1,2
0,46.962200,46.962200,46.962200
1,44.264867,44.264867,44.264867
2,43.331867,43.331867,43.331867
3,43.193400,43.193400,43.193400
4,46.683600,46.683600,46.683600
...,...,...,...
145,69.063267,69.063267,69.063267
146,64.445667,64.445667,64.445667
147,67.894600,67.894600,67.894600
148,68.116133,68.116133,68.116133


In [31]:
h["ones"] = 1

In [32]:
h

,0,1,2,ones
0,46.962200,46.962200,46.962200,1
1,44.264867,44.264867,44.264867,1
2,43.331867,43.331867,43.331867,1
3,43.193400,43.193400,43.193400,1
4,46.683600,46.683600,46.683600,1
...,...,...,...,...
145,69.063267,69.063267,69.063267,1
146,64.445667,64.445667,64.445667,1
147,67.894600,67.894600,67.894600,1
148,68.116133,68.116133,68.116133,1


In [36]:
z = h.dot(w2)

In [42]:
# Average Loss over batch
Lb = 0.5*((z-iris["petal_width"])**2)
(1/len(Lb))*np.sum(Lb)

64576.73855513819